In [1]:
import os

from cnn_finetune import make_model
import pandas as pd
from skimage import io
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data import Dataset
from torchvision import transforms, utils

https://pytorch.org/tutorials/beginner/data_loading_tutorial.html#dataset-class

In [2]:
model = make_model('resnet18', num_classes=2, pretrained=True, input_size=(128, 128))
device = torch.device('cpu')
model = model.to(device)

In [3]:
class MyDataSet(Dataset):
    def __init__(self, csv_path, root_dir, transform=None):
        self.data_df = pd.read_csv(csv_path)
        self.root_dir = root_dir
        
        if transform is None:
            self.transform = transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize(
                mean=model.original_model_info.mean,
                std=model.original_model_info.std),
            ])
        else:
            self.transform = transform
        
    def __len__(self):
        return self.data_df.shape[0]
    
    def __getitem__(self, idx):
        image_name = os.path.join(self.root_dir, self.data_df.iloc[idx, 0])
        image = io.imread(image_name)
        label = self.data_df.iloc[idx, 1]
        return self.transform(image), int(label)
    
    def get_full_data(self):
        image_names = [
            os.path.join(self.root_dir, self.data_df.iloc[idx, 0]) for idx in range(self.data_df.shape[0])
        ]
        images = [io.imread(path) for path in image_names]
        return images

In [4]:
train_set = MyDataSet('../images/train_data.csv', '../images/resize/data/')
train_loader = torch.utils.data.DataLoader(train_set, shuffle=True)

In [5]:
test_set = MyDataSet('../images/test_data.csv', '../images/resize/data/')
test_loader = torch.utils.data.DataLoader(test_set, shuffle=True)

https://github.com/creafz/pytorch-cnn-finetune/blob/master/examples/cifar10.py

In [6]:
criterion = nn.CrossEntropyLoss()
# optimizer = optim.Adam(model.parameters(), lr=0.01)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

In [7]:
def train(epoch):
    total_loss = 0
    total_size = 0
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        total_loss += loss.item()
        total_size += data.size(0)
        loss.backward()
        optimizer.step()
        if (batch_idx+1) % 20 == 0 or (batch_idx+1)==len(train_loader):
            progress = 100. * (batch_idx+1) / len(train_loader)
            average_loss = total_loss / total_size
            print(f'Train Epoch: {epoch} [{(batch_idx+1) * len(data)}/{len(train_loader.dataset)} ({progress:.0f}%)]\tAverage loss: {average_loss:.6f}')

In [8]:
def test():
    model.eval()
    test_loss = 0
    correct = 0
    outputs = []
    targets = []
    with torch.no_grad():
        for data, target in test_loader:
            targets.append(target)
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += criterion(output, target).item()
            pred = output.data.max(1, keepdim=True)[1]
            correct += pred.eq(target.data.view_as(pred)).long().cpu().sum().item()
            outputs.append(output.cpu().detach().numpy())

    test_loss /= len(test_loader.dataset)
    accuracy = 100. * correct / len(test_loader.dataset)
    print(f'\nTest set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)} ({accuracy:.0f}%)\n')
    return outputs, targets

In [9]:
for epoch in range(1, 6):
    train(epoch)
    test()

Train Epoch: 1 [20/173 (12%)]	Average loss: 5.133770
Train Epoch: 1 [40/173 (23%)]	Average loss: 7.711930
Train Epoch: 1 [60/173 (35%)]	Average loss: 6.867764
Train Epoch: 1 [80/173 (46%)]	Average loss: 5.515359
Train Epoch: 1 [100/173 (58%)]	Average loss: 5.176289
Train Epoch: 1 [120/173 (69%)]	Average loss: 4.552049
Train Epoch: 1 [140/173 (81%)]	Average loss: 4.091651
Train Epoch: 1 [160/173 (92%)]	Average loss: 3.767900
Train Epoch: 1 [173/173 (100%)]	Average loss: 3.560872

Test set: Average loss: 0.6735, Accuracy: 45/76 (59%)

Train Epoch: 2 [20/173 (12%)]	Average loss: 0.876862
Train Epoch: 2 [40/173 (23%)]	Average loss: 4.307245
Train Epoch: 2 [60/173 (35%)]	Average loss: 5.354260
Train Epoch: 2 [80/173 (46%)]	Average loss: 4.344302
Train Epoch: 2 [100/173 (58%)]	Average loss: 4.134697
Train Epoch: 2 [120/173 (69%)]	Average loss: 3.908905
Train Epoch: 2 [140/173 (81%)]	Average loss: 3.867714
Train Epoch: 2 [160/173 (92%)]	Average loss: 3.490555
Train Epoch: 2 [173/173 (100%)]	A